In [ ]:
import optuna
from pathlib import Path
import torch
import numpy as np
from datasets import Dataset
from transformers import (
    LlamaForCausalLM,
    LlamaTokenizer,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling
)
from peft import LoraConfig, get_peft_model, TaskType

In [ ]:
# 1) Prepara tokenizer e dati

tokenizer = LlamaTokenizer.from_pretrained("huggyllama/llama-7b")
if tokenizer.pad_token_id is None:
    tokenizer.add_special_tokens({"pad_token": "<pad>"})

# Carica dataset già tokenizzato (esempio)
# Supponiamo di avere `tokenized_ds` con split "train" e "test"
# tokenized_ds = your existing DatasetDict


In [ ]:
# 2) Definisci model_init()

def model_init():
    # carica base LLaMA
    model = LlamaForCausalLM.from_pretrained("huggyllama/llama-7b",
                                             torch_dtype=torch.float16)
    # applica LoRA
    lora_conf = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8, lora_alpha=16, lora_dropout=0.05,
        target_modules=["q_proj", "v_proj"]
    )
    return get_peft_model(model, lora_conf)

In [ ]:
# 3) Funzione di metriche

def compute_metrics(eval_pred):
    import numpy as np
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    mask  = labels != -100
    correct = (preds == labels) & mask
    acc = correct.sum() / mask.sum() if mask.sum()>0 else 0.0
    return {"accuracy": float(acc)}

In [ ]:
# 4) Spazio di ricerca (hp_space)

def hp_space(trial):
    return {
        "learning_rate":          trial.suggest_loguniform("learning_rate", 1e-5, 5e-4),
        "num_train_epochs":       trial.suggest_int("num_train_epochs", 1, 3),
        "per_device_train_batch_size": trial.suggest_categorical("per_device_train_batch_size", [1,2,4])
    }

In [ ]:
# 5) Crea Trainer con placeholder args

base_args = TrainingArguments(
    output_dir="hpo",
    eval_strategy="epoch",
    save_strategy="no",
    logging_strategy="epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=1,
    fp16=True,
    report_to="none"  # disabilita TensorBoard qui
)

trainer = Trainer(
    model_init=model_init,
    args=base_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset= tokenized_ds["test"],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    compute_metrics=compute_metrics
)


In [ ]:
# 6) Avvia hyperparameter_search

best_run = trainer.hyperparameter_search(
    direction="maximize",        # vogliamo massimizzare l'accuracy
    backend="optuna",
    hp_space=hp_space,
    n_trials=10                  # numero di esperimenti
)

print("Migliori iperparametri:", best_run.hyperparameters)
print("Best metric:", best_run.objective)